In [2]:
from datasets import load_dataset
import os
import torch
from sklearn.model_selection import train_test_split
from datasets import load_dataset, load_metric,Dataset,concatenate_datasets,set_caching_enabled, ClassLabel
import pandas as pd

import random
from IPython.display import display, HTML

import json
from transformers import Wav2Vec2CTCTokenizer,Wav2Vec2ForCTC,Wav2Vec2Processor,Trainer,TrainingArguments,Wav2Vec2FeatureExtractor

import re
set_caching_enabled(False)

import logging
import transformers
transformers.logging.get_verbosity = lambda: logging.NOTSET

transformers.logging.get_verbosity()


import soundfile as sf
import torchaudio


import IPython.display as ipd

import numpy as np
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

from tqdm import tqdm
import torch

# Set environment variables
os.environ['WANDB_DISABLED '] = 'True'

import warnings
warnings.filterwarnings('ignore')

In [3]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

processor = Wav2Vec2Processor.from_pretrained("prajin/wav2vec2-large-xlsr-300m-nepali")
model = Wav2Vec2ForCTC.from_pretrained("prajin/wav2vec2-large-xlsr-300m-nepali")

In [4]:
import datasets
datasets.logging.get_verbosity = lambda: logging.NOTSET

import pandas as pd

In [5]:
import glob 
data_dir  = "/workspace/data_processed/*"
data_dir_list = glob.glob(data_dir)
data_dir_list.remove('/workspace/data_processed/5')

In [6]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

processor = Wav2Vec2Processor.from_pretrained("prajin/wav2vec2-large-xlsr-300m-nepali")


In [7]:
data_set_list_total = []
for i in range(len(data_dir_list)):
    common_voice_val = Dataset.load_from_disk(data_dir_list[i])
    data_set_list_total.append(common_voice_val)


In [8]:
common_voice_test = concatenate_datasets(data_set_list_total[-1:])

In [9]:
common_voice_test

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 10000
})

In [10]:
label_str = processor.batch_decode(common_voice_test['labels'], group_tokens=False)

In [11]:
print(len(label_str))

10000


In [12]:
model = Wav2Vec2ForCTC.from_pretrained("prajin/wav2vec2-large-xlsr-300m-nepali")
" ".join(sorted(processor.tokenizer.get_vocab()))

'[PAD] [UNK] | ँ ं ः अ आ इ ई उ ऊ ऋ ए ऐ ओ औ क ख ग घ ङ च छ ज झ ञ ट ठ ड ढ ण त थ द ध न प फ ब भ म य र ल व श ष स ह ा ि ी ु ू ृ े ै ो ौ ् ॠ'

In [38]:
predictions_without_lm = []
import torch
for i in range(len(common_voice_test)):
    inputs = processor(common_voice_test[i]["input_values"], sampling_rate=16000, return_tensors="pt")

    with torch.no_grad():
        logits = model(**inputs).logits
        
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)
    predictions_without_lm.append(transcription[0].lower())
    if(i%500) == 0:
        print(i)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500


In [40]:
import pickle
with open('predictions_without_lm.pkl', 'wb') as f:
    pickle.dump(predictions_without_lm, f)

In [39]:
len(predictions_without_lm)

10000

In [43]:
wer = datasets.load_metric("wer")
wer_score = wer.compute(predictions=predictions_without_lm, references=label_str)
print(wer_score)

0.36003994341349754


In [44]:
# import torch

# with torch.no_grad():
#     logits = model(**inputs).logits

In [45]:
# predicted_ids = torch.argmax(logits, dim=-1)
# transcription = processor.batch_decode(predicted_ids)

# transcription[0].lower()

# With LM decoder

In [81]:
!pip install https://github.com/kpu/kenlm/archive/master.zip pyctcdecode
!pip install git+https://github.com/huggingface/transformers.git
!pip install datasets torchaudio
import pyctcdecode

  Using cached https://github.com/kpu/kenlm/archive/master.zip
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-ej4rdetd
  Running command git clone --filter=blob:none -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-ej4rdetd
  Resolved https://github.com/huggingface/transformers.git to commit f380bf2b612e6030ef8bc8904b287d274f035e29
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.17.0.dev0-py3-none-any.whl size=3598565 sha256=90adb01707b25e893407aea15adac73aab54ae831205d34f8a67ab3e0b904753
  Stored in directory: /tmp/pip-ephem-wheel-cache-20ohlboi/wheels/05/0a/97/64ae47c27ba95fae2cb5838e7b4b7247a34d4a8ba5f7092de2
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.16.0.dev0
    Uninstall

In [14]:
print("H")

H


In [15]:
common_voice_test

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 10000
})

In [17]:
# common_voice_test.save_to_disk("/workspace/test_data")

In [ ]:
from transformers import Wav2Vec2ProcessorWithLM
processor = Wav2Vec2ProcessorWithLM.from_pretrained("prajin/wav2vec2-large-xlsr-300m-nepali")

In [ ]:
processor.tokenizer

In [ ]:

" ".join(sorted(processor.tokenizer.get_vocab()))

In [ ]:
import torch
predictions_with_lm_0_1000 = []
for i in range(1000):
    inputs = processor(common_voice_test[i]["input_values"], sampling_rate=16000, return_tensors="pt")

    with torch.no_grad():
        logits = model(**inputs).logits
        
    transcription = processor.batch_decode(logits.numpy()).text        
    predictions_with_lm.append(transcription[0].lower())
    if(i<100):
        print(i)
    if(i%500) == 0:
        print(i)
    

0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


In [26]:
wer = datasets.load_metric("wer")
wer_score = wer.compute(predictions=predictions_with_lm, references=label_str)
print(wer_score)

0.37681159420289856
